In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import copy
import random

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix
from torch.quantization import QuantStub, DeQuantStub

# Статическая квантизация
Статическая квантизация позволяет сразу все операции перевести в int, без необходимости дополнительно что-то расчитывать в процессе предсказания.

По сравнению с моделью из главы про разряжение нейронной сети, архитектура повлекла небольшие изменения. В частности так как квантизация не происходит динамически, необходимо дополнительно руками квантовать входные данные и деквантовать ответ. Это можно видеть в `forward` методе. 

Также стоит обратить внимание на применение ` nn.quantized.FloatFunctional()` при выполнении skip connection в ResNet архитектуре. Это необходимо для правильного выполнения операции сложения со сквантованным входом и весами в представлении float32

In [ ]:
BATCH_SIZE = 32
EPOCH = 6
DEVICE = 'cuda'
SEED = 5

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, quant_func, dequant_func, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.skip_add = nn.quantized.FloatFunctional()
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
        self.quant = quant_func
        self.dequant = dequant_func

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        skip_branch = self.shortcut(x)
        out = self.skip_add.add(out, skip_branch)
        out = F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        block = BasicBlock
        num_blocks = [2, 2, 2, 2]
        self.in_planes = 64
        self.quant = QuantStub()
        self.dequant = DeQuantStub()

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, self.quant, self.dequant, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.quant(x)
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = self.dequant(out)
        return out

In [ ]:
def fit(model, train_loader, epoch_number=5, device='cuda'):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    model.train()
    
    for epoch in range(epoch_number):
        correct = 0
        
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = X_batch.to(device)
            var_y_batch = y_batch.to(device)
            
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 500 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 
                    100.*batch_idx / len(train_loader), loss.data, 
                    float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))
                
                
def evaluate(model, loader, device='cuda'):
    correct = 0
    model.eval() 
    for test_imgs, test_labels in loader:
        test_imgs = test_imgs.to(device)
        test_labels = test_labels.to(device)
        
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("Test accuracy:{:.3f}% ".format( float(correct) / (len(loader)*BATCH_SIZE)))

    
def calc_size(model):
    torch.save(model.state_dict(), "/tmp/model.p")
    size=os.path.getsize("/tmp/model.p")
    os.remove('/tmp/model.p')
    return "{:.3f} KB".format(size / 1024)


Подготовим данные для обучения

In [ ]:
train_data = torchvision.datasets.FashionMNIST('./', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_data = torchvision.datasets.FashionMNIST('./', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
train_loader = torch.utils.data.DataLoader(train_data, batch_size = BATCH_SIZE, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = BATCH_SIZE, shuffle = True)

Статическое квантование после обучения включает в себя не только преобразование весов из float32 в int, как при динамическом квантовании, но также выполнение дополнительного шага первоначальной прогонки выборки обучающих данных через сеть и вычисления результирующих распределений различных активаций (в частности, это выполняется путем вставки модулей, так называемого наблюдателя, в нужные места после каждой операции, которые записывают эти данные). Эти распределения затем используются для определения того, как конкретно различные активации должны быть сквантованы во время вывода (вычисляется свой коэффициент масштабирования и смещения). Важно отметить, что этот дополнительный шаг позволяет нам передавать квантованные значения между операциями вместо преобразования этих значений в числа с плавающей запятой - а затем обратно в целые числа - между каждой операцией, что приводит к значительному ускорению.

In [ ]:
resnet = ResNet()
resnet.to(DEVICE)
torch.manual_seed(SEED)
fit(resnet, train_loader, epoch_number=EPOCH, device=DEVICE)

Epoch : 0 [0/60000 (0%)]	Loss: 2.333537	 Accuracy:9.375%
Epoch : 0 [16000/60000 (27%)]	Loss: 0.385268	 Accuracy:74.613%
Epoch : 0 [32000/60000 (53%)]	Loss: 0.373411	 Accuracy:79.661%
Epoch : 0 [48000/60000 (80%)]	Loss: 0.338566	 Accuracy:82.535%
Epoch : 1 [0/60000 (0%)]	Loss: 0.168826	 Accuracy:96.875%
Epoch : 1 [16000/60000 (27%)]	Loss: 0.335091	 Accuracy:90.151%
Epoch : 1 [32000/60000 (53%)]	Loss: 0.074745	 Accuracy:90.207%
Epoch : 1 [48000/60000 (80%)]	Loss: 0.176288	 Accuracy:90.408%
Epoch : 2 [0/60000 (0%)]	Loss: 0.195265	 Accuracy:90.625%
Epoch : 2 [16000/60000 (27%)]	Loss: 0.140607	 Accuracy:91.935%
Epoch : 2 [32000/60000 (53%)]	Loss: 0.321993	 Accuracy:91.868%
Epoch : 2 [48000/60000 (80%)]	Loss: 0.064194	 Accuracy:91.920%
Epoch : 3 [0/60000 (0%)]	Loss: 0.210873	 Accuracy:90.625%
Epoch : 3 [16000/60000 (27%)]	Loss: 0.084497	 Accuracy:93.569%
Epoch : 3 [32000/60000 (53%)]	Loss: 0.244260	 Accuracy:93.300%
Epoch : 3 [48000/60000 (80%)]	Loss: 0.294583	 Accuracy:93.269%
Epoch : 4 [0/

Мы Натренировали изначальную сеть на данных FashionMNIST, теперь изерим качество. Это будет нашим бейзлайном.

In [ ]:
evaluate(resnet, test_loader, device=DEVICE)

Test accuracy:0.925% 


Полученное качество составляет 92.5% на валидационной выборке. Далее замерим время инференса сети на ЦПУ. Чтобы замер был честный, отключим возможность PyTorch использовать несколько потоков и будем использовать всего один поток вычислений


In [ ]:
from contextlib import contextmanager

@contextmanager
def single_thread():  
    num = torch.get_num_threads()
    torch.set_num_threads(1)
    yield
    torch.set_num_threads(num)

In [ ]:
%%timeit -r5
resnet.to('cpu')
with single_thread():
    evaluate(resnet, test_loader, device='cpu')

Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
1 loop, best of 5: 6min 31s per loop


Видим, что скорость инференса сети в среднем 6 минут и 31 секунда. Измерим вес сети, занимаемой памяти в хранилище данных. Она составляет примерно 44 мб памяти. Теперь применим алгоритмы квантизации и попробуем уменьшить это значение в несколько раз, при этом не потеряв сильно в качестве.

In [ ]:
calc_size(resnet)

'43722.743 KB'

Для моделей мы также можем указать конфиг квантования, где в частности можно указать библиотеку для работы с квантованными значениями. Далее устанавливаем модули подсчета параметров квантования. По умолчанию исползуется HistogramObserver, это модуль, который рассчтывает параметры на основе гистрограммы распределения значнеий для конкретного слоя

In [ ]:
resnet.qconfig = torch.quantization.get_default_qconfig('fbgemm')

In [ ]:
torch.quantization.prepare(resnet, inplace=True)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:123: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


ResNet(
  (quant): QuantStub(
    (activation_post_process): HistogramObserver()
  )
  (dequant): DeQuantStub()
  (conv1): Conv2d(
    1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
    (activation_post_process): HistogramObserver()
  )
  (bn1): BatchNorm2d(
    64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (activation_post_process): HistogramObserver()
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (activation_post_process): HistogramObserver()
      )
      (bn1): BatchNorm2d(
        64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
        (activation_post_process): HistogramObserver()
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (activation_post_process): HistogramObserver()
      )
      (bn2): BatchNorm2d(
        64, eps=1e-05, momentu

Прогоняем всю обучающую выборку через сеть.
Само значение нам не интересно, нам важно, чтобы посчитались параметры

In [ ]:
resnet.to(DEVICE)
evaluate(resnet, train_loader, device=DEVICE)

Test accuracy:0.968% 


Фиксируем полученные веса и параметры квантизации

In [ ]:
resnet.cpu()
torch.quantization.convert(resnet, inplace=True)

ResNet(
  (quant): Quantize(scale=tensor([0.0079]), zero_point=tensor([0]), dtype=torch.quint8)
  (dequant): DeQuantize()
  (conv1): QuantizedConv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.02345956489443779, zero_point=63, padding=(1, 1), bias=False)
  (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.4392222762107849, zero_point=76, padding=(1, 1), bias=False)
      (bn1): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): QuantizedConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.33188363909721375, zero_point=74, padding=(1, 1), bias=False)
      (bn2): QuantizedBatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (skip_add): QFunctional(
        scale=0.15338747203350067, zero_point=52
        (activation_post_

Посчитаем теперь размер нашей сети. Применяя алгоритм квантизации, удалось сжать ее размер примерно в 4 раза. Теперь вместо  44 мб. она занимает всего 11 мб.

In [ ]:
calc_size(resnet)

'11128.655 KB'

Можно видеть, что теперь все веса имеют свой коэффициент масштабирования и смещения, а также увидеть int8 представление на примере линейного слоя

In [ ]:
print(resnet.conv1.weight)
resnet.linear.weight().int_repr()

<bound method Conv2d.weight of QuantizedConv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), scale=0.02345956489443779, zero_point=63, padding=(1, 1), bias=False)>


tensor([[   4,   -5,   -5,  ...,   48,  -13,  -16],
        [ -78,   -4,   -9,  ...,   37,  -29,    3],
        [ -58,  -30,  -95,  ..., -127,  -14,   20],
        ...,
        [   7,   15,   12,  ...,   15,  -17,  -13],
        [ -28,  -79,   25,  ...,  -43,   70,   18],
        [ -36,   20,   15,  ...,  -50,    9,    3]], dtype=torch.int8)

По результатам тестирования можно сделать вывод, что просадки в точности сети нет, покрайней мере на наших тестовых данных удалось достичь метрики оригинальной модели

In [ ]:
resnet.to('cpu')
evaluate(resnet, test_loader, device='cpu')

Test accuracy:0.925% 


Интересно сравнить, насколько наша сеть стала быстрее по сравнению с оригинальной

In [ ]:
%%timeit -r5

with single_thread():
    evaluate(resnet, test_loader, device='cpu')

Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
Test accuracy:0.925% 
1 loop, best of 5: 3min 52s per loop


## Квантизация в процессе обучения

Этот метод заключается в том, что квантование происходит на каждом шаге градиентного спуска. С QAT (Quantization-aware training) все веса и активации «поддельно квантуются» во время как прямого, так и обратного проходов обучения: то есть числа с плавающей точкой округляются до имитации значений int8, но все вычисления по-прежнему выполняются во float32 представлении. Таким образом, все корректировки веса во время обучения производятся с учетом того факта, что модель в конечном итоге будет квантована; поэтому после квантования этот метод обычно дает более высокую точность, чем динамическое квантование или статическое квантование после обучения.

Общий рабочий процесс для фактического выполнения QAT очень похож на предыдущий:

Мы можем использовать ту же модель, что и раньше: для обучения с учетом квантования не требуется дополнительной подготовки.
Нам нужно использовать qconfig, указывающий, какой тип фальшивого квантования должен быть вставлен после весов и активаций, вместо указания модулей наблюдателей (Histogram observers).

Добавляем конфигурацию, после чего подготавливаем модель для обучения с квантованием

Модель внутри себя автоматически будет обновлять веса с учетом квантования

In [ ]:
qa_resnet = ResNet()
qa_resnet.to(DEVICE)
qa_resnet.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')
torch.quantization.prepare_qat(qa_resnet, inplace=True)
torch.manual_seed(SEED)
fit(qa_resnet, train_loader, epoch_number=10)

/usr/local/lib/python3.7/dist-packages/torch/quantization/observer.py:123: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  reduce_range will be deprecated in a future release of PyTorch."


Epoch : 0 [0/60000 (0%)]	Loss: 2.343018	 Accuracy:6.250%
Epoch : 0 [16000/60000 (27%)]	Loss: 0.299412	 Accuracy:76.659%
Epoch : 0 [32000/60000 (53%)]	Loss: 0.466928	 Accuracy:80.975%
Epoch : 0 [48000/60000 (80%)]	Loss: 0.425663	 Accuracy:83.405%
Epoch : 1 [0/60000 (0%)]	Loss: 0.144397	 Accuracy:96.875%
Epoch : 1 [16000/60000 (27%)]	Loss: 0.345503	 Accuracy:90.525%
Epoch : 1 [32000/60000 (53%)]	Loss: 0.087875	 Accuracy:90.325%
Epoch : 1 [48000/60000 (80%)]	Loss: 0.156576	 Accuracy:90.417%
Epoch : 2 [0/60000 (0%)]	Loss: 0.311665	 Accuracy:87.500%
Epoch : 2 [16000/60000 (27%)]	Loss: 0.117227	 Accuracy:92.072%
Epoch : 2 [32000/60000 (53%)]	Loss: 0.214537	 Accuracy:91.899%
Epoch : 2 [48000/60000 (80%)]	Loss: 0.109145	 Accuracy:92.009%
Epoch : 3 [0/60000 (0%)]	Loss: 0.104612	 Accuracy:96.875%
Epoch : 3 [16000/60000 (27%)]	Loss: 0.080911	 Accuracy:93.438%
Epoch : 3 [32000/60000 (53%)]	Loss: 0.292502	 Accuracy:93.316%
Epoch : 3 [48000/60000 (80%)]	Loss: 0.255406	 Accuracy:93.294%
Epoch : 4 [0/

После обучения с квантованием, фиксируем квантованные веса и параметры. Таким образом получаем финальную сеть для последующего использования

In [ ]:
qa_resnet.to('cpu')
quantized_model = torch.quantization.convert(qa_resnet, inplace=False)
evaluate(quantized_model, test_loader, device='cpu')

Test accuracy:0.925% 


Видим, что нам удалось все также уменьшить вес сети, как и при статической квантизации

In [ ]:
calc_size(quantized_model)

'11128.655 KB'

Таким образом происходит применение алгоритмов квантизации на практике. Как видно из примера, удалось эффективно перевести веса модели ResNet18 в int8 представление, тем самым уменьшить требуемую память для хранения сети и значительно ускорить пропускную способность на процессоре.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp "drive/My Drive/Colab Notebooks/quantization.ipynb" ./

!jupyter nbconvert --to latex quantization.ipynb


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[NbConvertApp] Converting notebook quantization.ipynb to latex
[NbConvertApp] Writing 79478 bytes to quantization.tex


In [ ]:
!cp quantization.tex "drive/My Drive/Colab Notebooks/"